In [1]:
# Misc imports
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')
import torch
torch.set_printoptions(linewidth=200)
import seaborn as sns
sns.set_style("darkgrid")
import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
       '/storage'
))
import yaml
from pathlib import Path
# Local imports
from ese.experiment.analysis.run_inference import get_cal_stats
from ese.experiment.analysis.analysis_utils.submit_utils import get_ese_inference_configs
# Ionpy imports
from ionpy.util import Config
from ionpy.analysis import ResultsLoader

# Define some useful paths.
scratch_root = Path("/storage/vbutoi/scratch/ESE")
code_root = Path("/storage/vbutoi/projects/ESE")
rs = ResultsLoader()

# For using code without restarting.
%load_ext autoreload
%autoreload 2
# For using yaml configs.
%load_ext yamlmagic

/storage/vbutoi/libraries/pylot/pandas/register.py:47: UserWarning: registration of accessor <class 'pylot.pandas.register.register_series_method.<locals>.inner.<locals>.AccessorMethod'> under name 'fillNA' for type <class 'pandas.core.series.Series'> is overriding a preexisting attribute with the same name.
  register_series_accessor(method.__name__)(AccessorMethod)
/storage/vbutoi/libraries/pylot/pandas/register.py:47: UserWarning: registration of accessor <class 'pylot.pandas.register.register_series_method.<locals>.inner.<locals>.AccessorMethod'> under name 'isNA' for type <class 'pandas.core.series.Series'> is overriding a preexisting attribute with the same name.
  register_series_accessor(method.__name__)(AccessorMethod)
/storage/vbutoi/libraries/pylot/pandas/register.py:47: UserWarning: registration of accessor <class 'pylot.pandas.register.register_series_method.<locals>.inner.<locals>.AccessorMethod'> under name 'notNA' for type <class 'pandas.core.series.Series'> is overridi

In [2]:
%%yaml inference_config

experiment:
    exp_root: '?'
    threshold: '?' 
    seed: 42
    supports_per_target: 5
    fixed_support_sets: True 

log:
    root: '?'
    min_fg_pixels: 0 
    log_interval: 20 
    log_image_stats: True 
    log_pixel_stats: True 
    gether_inference_stats: True
    track_ensemble_member_scores: False 
    summary_compute_global_metrics: True

dataloader:
    batch_size: 1 
    num_workers: 0
    pin_memory: True 

<IPython.core.display.Javascript object>

In [3]:
%%yaml calibration_cfg 

calibrator:
    _name: '?'
    normalize: False

local_calibration:
    num_prob_bins: 15
    neighborhood_width: 3
    square_diff: False 

global_calibration:
    num_prob_bins: 15
    num_classes: 2
    neighborhood_width: 3
    square_diff: False 
    loss_weights: None 

<IPython.core.display.Javascript object>

In [4]:
%%yaml incontext_model_cfg 

# # For incontext datasets
####################################
data:
    splits: ('val',)
    support_size: '?' 
    preload: False 

ensemble:
    combine_fn: '?'
    combine_quantity: '?'
    normalize: False

model:
    _type: "incontext"
    pretrained_exp_root : None
    checkpoint: "max-val_od-dice_score"
    # _attr:
    #     model: 'universeg.experiment.models.QCUNet3'
    #     data_support_size: 64
    #     mix_filters: (64, 64, 64, 64)

<IPython.core.display.Javascript object>

In [5]:
%%yaml incontext_submit_cfg_args

submit_opts:
    ens_cfg_options:
        - ('mean', 'probs')
    num_ens_membs:
        - 1
        # - 2
        # - 4
        # - 8
        # - 16
        # - 32

<IPython.core.display.Javascript object>

In [6]:
exp_name = f'04_24_24_UniverSeg_VaryThreshold'
group_dict = {
    "datasets": [
        "ACDC", 
        # "PanDental_Jaw", 
        # "PanDental_Mandible", 
        # "SpineWeb", 
        # "STARE", 
        # "WBC"
    ],
    "exp_group": exp_name,
    "model_type": "incontext",
    "base_models_group": "/storage/jjgo/results/omni/2023-02-16_Universeg-HO_Adrian/20230214_112221-1C7K-2cb972b55f72d56b3842c2d2d8bdd3c6",
    "scratch_root": "/storage/vbutoi/scratch/ESE",
    "inf_cfg_root": "/storage/vbutoi/projects/ESE/ese/experiment/configs/inference",
}

## Gather Inference Options.

In [7]:
%%yaml inference_config_opts

calibrator:
    - Uncalibrated
    # - ContextualHistogramBinning 

data.support_size:
    - 8

experiment.threshold:
    - 0.1
    - 0.2
    - 0.3
    - 0.4
    - 0.5
    - 0.6
    - 0.7
    - 0.8
    - 0.9

<IPython.core.display.Javascript object>

In [8]:
# Load the inference cfg from local.
##################################################
inf_cfg_root = code_root / "ese" / "experiment" / "configs" / "inference"

##################################################
with open(inf_cfg_root / "Calibration_Metrics.yaml", 'r') as file:
    cal_metrics_cfg = yaml.safe_load(file)

##################################################
base_cfg = Config(inference_config).update([calibration_cfg, cal_metrics_cfg, incontext_model_cfg])

In [9]:
inference_func = get_cal_stats
# Get the configs for the different runs.
inf_cfgs = get_ese_inference_configs(
    group_dict=group_dict,
    base_cfg=base_cfg,
    inf_cfg_opts=inference_config_opts,
    base_cfg_args=incontext_submit_cfg_args
)

WARNING! Calibrator not found, using default calibrator name: Uncalibrated.


In [10]:
len(inf_cfgs)

7

## Running Jobs

In [11]:
from ese.experiment.analysis.run_inference import get_cal_stats

In [12]:
# from ese.experiment.experiment import run_ese_exp

# ###### Run individual jobs
# run_ese_exp(
#     config=inf_cfgs[0], 
#     job_func=get_cal_stats,
#     run_name='debug',
#     show_examples=True,
#     gpu='3',
# ) 

In [13]:
from ese.experiment.experiment import submit_ese_exps 

#### Run Batch Jobs
submit_ese_exps(
    config_list=inf_cfgs,
    job_func=get_cal_stats,
    available_gpus=['0', '1', '2', '3']
)

Submitted job id: 2317617 on gpu: 0.
Submitted job id: 2317696 on gpu: 1.
Submitted job id: 2317816 on gpu: 2.
Submitted job id: 2317933 on gpu: 3.
Submitted job id: 2318066 on gpu: 0.
Submitted job id: 2318182 on gpu: 1.
Submitted job id: 2318298 on gpu: 2.
